# Extrator de dados do Sweet Coffee Week - Natal (2021)

## Importação de libs

In [1]:
import mechanize
import pandas as pd
import bs4
import requests
from bs4 import BeautifulSoup
import re
from concurrent import futures
from urllib.parse import urlparse

## Extração de links das páginas internas das cafeterias

In [2]:
brw = mechanize.Browser()

In [3]:
pag = brw.open('https://www.sweetcoffeeweek.com.br/')

In [4]:
soup = BeautifulSoup(pag.read())

In [5]:
a_urls = soup.find_all('a', {'href': re.compile(r'https://www\.sweetcoffeeweek\.com\.br/\w+')})[:-1]
urls = list(set([link['href'] for link in a_urls]))

In [6]:
urls

['https://www.sweetcoffeeweek.com.br/mrcupcake',
 'https://www.sweetcoffeeweek.com.br/casadossalgados',
 'https://www.sweetcoffeeweek.com.br/mangai',
 'https://www.sweetcoffeeweek.com.br/momentogourmet',
 'https://www.sweetcoffeeweek.com.br/paneer',
 'https://www.sweetcoffeeweek.com.br/ceciliamindelo',
 'https://www.sweetcoffeeweek.com.br/bocaditos',
 'https://www.sweetcoffeeweek.com.br/cassiaribeiro',
 'https://www.sweetcoffeeweek.com.br/cafebrigadeiro',
 'https://www.sweetcoffeeweek.com.br/marlonvinicius',
 'https://www.sweetcoffeeweek.com.br/bells',
 'https://www.sweetcoffeeweek.com.br/bellapetit',
 'https://www.sweetcoffeeweek.com.br/rafaelafontes',
 'https://www.sweetcoffeeweek.com.br/aumer',
 'https://www.sweetcoffeeweek.com.br/dasmelo',
 'https://www.sweetcoffeeweek.com.br/franscafe',
 'https://www.sweetcoffeeweek.com.br/radiocafe',
 'https://www.sweetcoffeeweek.com.br/chocolatudos',
 'https://www.sweetcoffeeweek.com.br/lulu',
 'https://www.sweetcoffeeweek.com.br/cacaushow',
 'h

## Extração de dados de menu das cafeterias

In [7]:
def baixar_dados_cafeterias(url):
    print(url, '⏳')
    page = brw.open(url)
    soup = BeautifulSoup(page.read())
    print(url, '✅')
    return soup


with futures.ThreadPoolExecutor(max_workers=4) as executor:
    threads = [executor.submit(baixar_dados_cafeterias, url) for url in urls]
    futures.wait(threads, timeout=120)  

https://www.sweetcoffeeweek.com.br/mrcupcakehttps://www.sweetcoffeeweek.com.br/casadossalgados ⏳
 ⏳
https://www.sweetcoffeeweek.com.br/mangai ⏳
https://www.sweetcoffeeweek.com.br/momentogourmet ⏳
https://www.sweetcoffeeweek.com.br/momentogourmet ✅
https://www.sweetcoffeeweek.com.br/paneer ⏳
https://www.sweetcoffeeweek.com.br/casadossalgados ✅
https://www.sweetcoffeeweek.com.br/ceciliamindelo ⏳
https://www.sweetcoffeeweek.com.br/mangai ✅
https://www.sweetcoffeeweek.com.br/bocaditos ⏳
https://www.sweetcoffeeweek.com.br/ceciliamindelo ✅
https://www.sweetcoffeeweek.com.br/cassiaribeiro ⏳
https://www.sweetcoffeeweek.com.br/mrcupcake ✅
https://www.sweetcoffeeweek.com.br/cafebrigadeiro ⏳
https://www.sweetcoffeeweek.com.br/bocaditos ✅
https://www.sweetcoffeeweek.com.br/marlonvinicius ⏳
https://www.sweetcoffeeweek.com.br/paneer ✅
https://www.sweetcoffeeweek.com.br/bells ⏳
https://www.sweetcoffeeweek.com.br/cafebrigadeiro ✅
https://www.sweetcoffeeweek.com.br/bellapetit ⏳
https://www.sweetcoffeew

In [8]:
dados_cafeterias = []
re_pratos = re.compile(r'(?P<doce>(?<=DOCE:).+$)|(?P<salgado>(?<=SALGADO:).+$)|(?P<bebida>(?<=BEBIDA:).+$)', 
                       re.MULTILINE)

for t in threads:
    soup = t.result()
    nome = soup.span.text
    prato = '\n'.join([x.text for x in soup.find_all('h1')[1:-1]])
    dicio = {
        'nome': soup.span.text.upper(),
        'serie': soup.h1.text[17:].strip().upper(),
    }
    pos = 0
    while r:=re_pratos.search(prato, pos=pos):
        pos = r.end()+1
        dicio |= {k:v.strip() for k,v in r.groupdict().items() if v is not None}
        
    dados_cafeterias.append(dicio)

## Resultado da extração

In [9]:
df_cafeterias = pd.DataFrame(dados_cafeterias)

In [10]:
df_cafeterias

,nome,serie,doce,salgado,bebida
0,MR. CUPCAKE,GAME OF THRONES,"Mr. Cupcake Trono de Ferro, composto de um Cup...",Mr. Coxinha de Frango.,Mr. Cappuccino Hidromel.
1,CASA DOS SALGADOS,VIKINGS,Torta Folhada de Maçã.,Trio de coxinhas bárbaras (sabores inusitados).,"Café, Refrigerante ou Suco de Laranja."
2,MANGAI,THE CROWN,Pão doce de macaxeira doce com Nutella.,Tapioca com carne de sol e nata.,Café coado.
3,MOMENTO GOURMET,GILMORE GIRLS,Waffle.,Coxinha de Costela.,Cappuccino.
4,PANEER,CHAVES,Sonho de churros.,Sanduíche de presunto no croissant.,"Café, café com leite ou água."
5,CECILIA MINDÊLO,ANNE WITH AN E,Bolo de aniversário esplêndido de Anne with an E.,Toast green gables recheado com creme especial...,Café ou água.
6,BOCADITOS,O GAMBITO DA RAINHA,TORTA MEDOVíK. Elaborada em várias camadas com...,Pirozhki. Um salgado assado semelhante a um pa...,café expresso ou água ou chá Twinings (algo mu...
7,CÁSSIA RIBEIRO CONFEITARIA,GOSSIP GIRL,Redvelvet Devil’s cake.,"Hot dog, pão brioche, salsicha, bacon e salada...",Pink Lemonade ou Mini Frappuccino.
8,CAFÉ BRIGADEIRO,QUEM MATOU SARA?,Deliciosa sobremesa de palha italiana e chocol...,Irresistível coxinha de frango servida com mol...,Cappuccino gelado 100 ml (sorvete e chantininh...
9,MARLON VINICIUS GASTRONOMIA,CIDADE INVISÍVEL,"Cuca de Banana: massa fofinha com banana, fina...","Pão de Milho Artesanal, recheado com carne cre...",Café Expresso (Pode ser trocado por Refrigeran...
